# 17/05 - Pair Programming ETL 1 - Lidia y Tatiana

Tendréis que usar el csv attacks_limpieza_completa.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD.
 
En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora. Usad el que no tenía las variables categóricas codificadas.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos

Del dataframe de los ataques nos quedaremos solo con las filas de los paises que seleccionamos en la lección de ayer:

>USA

>Australia

>New Zealand

>South Africa

>Papua New Guinea

Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por pais.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 

Si intentamos hacer la media de eso no nos dará un valor real. 

Como separar esta información en distintas columnas fue un problema al que nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. 

Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [2]:
# Cargamos el primer archivo
df = pd.read_csv("./attacks_limpieza_completa.csv", index_col = 0)
df.reset_index(inplace=True)

In [4]:
df.head(1)

,index,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,...,1,0,0,1,0,0,4,7,1,9.116327


In [5]:
# Del primer archivo seleccionamos solo los paises que nos interesan
df_paises= df[df["country"].isin(["usa","australia","new zealand", "south africa", "papua new guinea"])]

In [6]:
# Cargamos el segundo archivo
df_clima= pd.read_csv("./df_completo.csv", index_col = 0)
df_clima.reset_index(inplace=True)

In [7]:
df_clima.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 245, 'speed':...",21,2,8,1017,none,0,0,200,2,USA,39.78373,-100.445882


In [8]:
import ast

In [9]:
df_clima["rh_profile"] =df_clima["rh_profile"].apply(ast.literal_eval)

In [10]:
df_clima.rh_profile[0] 

[{'layer': '950mb', 'rh': 5},
 {'layer': '900mb', 'rh': 4},
 {'layer': '850mb', 'rh': 4},
 {'layer': '800mb', 'rh': 4},
 {'layer': '750mb', 'rh': 3},
 {'layer': '700mb', 'rh': 1},
 {'layer': '650mb', 'rh': 2},
 {'layer': '600mb', 'rh': 3},
 {'layer': '550mb', 'rh': 4},
 {'layer': '500mb', 'rh': 5},
 {'layer': '450mb', 'rh': 1},
 {'layer': '400mb', 'rh': 0},
 {'layer': '350mb', 'rh': 3},
 {'layer': '300mb', 'rh': 2},
 {'layer': '250mb', 'rh': 3},
 {'layer': '200mb', 'rh': 8}]

In [11]:
rh= df_clima["rh_profile"].apply(pd.Series)

In [12]:
for i in range(len(rh.columns)):
    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    valores = list(rh[i].apply(pd.Series)["rh"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [13]:
df_clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,5,4,4,4,3,1,2,3,4,5,...,8,1017,none,0,0,200,2,USA,39.78373,-100.445882
1,5,4,4,4,1,0,2,3,4,3,...,8,1018,none,0,0,230,2,USA,39.78373,-100.445882


In [14]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [15]:
df_clima['wind_profile'][0] 

[{'layer': '950mb', 'direction': 245, 'speed': 2},
 {'layer': '900mb', 'direction': 230, 'speed': 3},
 {'layer': '850mb', 'direction': 240, 'speed': 3},
 {'layer': '800mb', 'direction': 255, 'speed': 2},
 {'layer': '750mb', 'direction': 265, 'speed': 3},
 {'layer': '700mb', 'direction': 265, 'speed': 3},
 {'layer': '650mb', 'direction': 255, 'speed': 3},
 {'layer': '600mb', 'direction': 250, 'speed': 4},
 {'layer': '550mb', 'direction': 260, 'speed': 5},
 {'layer': '500mb', 'direction': 270, 'speed': 5},
 {'layer': '450mb', 'direction': 285, 'speed': 4},
 {'layer': '400mb', 'direction': 290, 'speed': 5},
 {'layer': '350mb', 'direction': 290, 'speed': 5},
 {'layer': '300mb', 'direction': 300, 'speed': 6},
 {'layer': '250mb', 'direction': 305, 'speed': 5},
 {'layer': '200mb', 'direction': 285, 'speed': 5}]

In [16]:
direction= df_clima["wind_profile"].apply(pd.Series)

In [17]:
for i in range(len(direction.columns)):
    nombre = "wind_" + str(direction[i].apply(pd.Series)["layer"][0]) 
    valores = list(direction[i].apply(pd.Series)["direction"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [18]:
speed= df_clima["wind_profile"].apply(pd.Series)

In [19]:
for i in range(len(speed.columns)):
    nombre = "wind_speed_" + str(speed[i].apply(pd.Series)["layer"][0]) 
    valores = list(speed[i].apply(pd.Series)["speed"])#tenemos que convertirlo a lista para q funcione el insert
    df_clima.insert(i, nombre, valores)#insert acepta listas

In [20]:
df_clima.columns #verificamos que todas nuestras columnas esta creadas

Index(['wind_speed_950mb', 'wind_speed_900mb', 'wind_speed_850mb',
       'wind_speed_800mb', 'wind_speed_750mb', 'wind_speed_700mb',
       'wind_speed_650mb', 'wind_speed_600mb', 'wind_speed_550mb',
       'wind_speed_500mb', 'wind_speed_450mb', 'wind_speed_400mb',
       'wind_speed_350mb', 'wind_speed_300mb', 'wind_speed_250mb',
       'wind_speed_200mb', 'wind_950mb', 'wind_900mb', 'wind_850mb',
       'wind_800mb', 'wind_750mb', 'wind_700mb', 'wind_650mb', 'wind_600mb',
       'wind_550mb', 'wind_500mb', 'wind_450mb', 'wind_400mb', 'wind_350mb',
       'wind_300mb', 'wind_250mb', 'wind_200mb', 'rh_950mb', 'rh_900mb',
       'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb',
       'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb',
       'rh_250mb', 'rh_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud',
       'midcloud', 'lowcloud', 'rh_profile', 'wind_profile', 'temp2m',
       'lifted_index', 'rh2m', 'msl_pressure', 'prec_type', 

Sacamos las medias de la tabla de clima por pais

In [21]:
df_clima_agrupado= df_clima.groupby("pais").mean()

In [23]:
df_clima_agrupado.head(1)

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
pais,,,,,,,,,,,,,,,,,,,,,
Australia,3.125,2.921875,2.8125,2.640625,2.671875,2.78125,2.859375,3.09375,3.1875,3.359375,...,24.09375,0.375,10.796875,1019.15625,3.484375,0.0,133.4375,2.984375,-24.776109,134.755


In [24]:
df_clima_agrupado= pd.DataFrame(df_clima_agrupado.reset_index())

In [25]:
df_clima_agrupado.head(1)

,pais,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
0,Australia,3.125,2.921875,2.8125,2.640625,2.671875,2.78125,2.859375,3.09375,3.1875,...,24.09375,0.375,10.796875,1019.15625,3.484375,0.0,133.4375,2.984375,-24.776109,134.755


     Que para mergear mas adelante, necesitamos que "country" y "país" tengan estén escritos igual, y por ello cambiamos a minusculo la info de la columna country.

In [29]:
def minuscula(col):
    try:
        col=col.lower()
    except:
        pass
    return col

In [30]:
df_clima_agrupado["pais"] = df_clima_agrupado["pais"].apply(minuscula)

In [31]:
df_clima_agrupado["pais"].unique()

array(['australia', 'new zealand', 'papua new guinea', 'south africa',
       'usa'], dtype=object)

In [32]:
df_unido_total = pd.merge(df_paises,df_clima_agrupado, left_on=["country"], right_on=["pais"])

In [33]:
df_unido_total.head(4)

,index_x,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,...,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_y,longitud_y
0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
1,1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
2,2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882
3,7,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.78373,...,22.609375,1.90625,5.03125,1015.46875,0.0,0.0,136.09375,2.203125,39.78373,-100.445882


Guardar los resultados obtenidos en un csv.

In [34]:
df_unido_total.to_csv("df_unido.csv")